In [4]:
from elasticsearch import Elasticsearch 

In [5]:
# Connect to the elastic cluster
es=Elasticsearch([{'host':'localhost','port':9200}])
es

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [7]:
# documentos
e1={
    "first_name":"nitin",
    "last_name":"panwar",
    "age": 27,
    "about": "Love to play cricket",
    "interests": ['sports','music'],
}

e2={
    "first_name" :  "Jane",
    "last_name" :   "Smith",
    "age" :         32,
    "about" :       "I like to collect rock albums",
    "interests":  [ "music" ]
}
e3={
    "first_name" :  "Douglas",
    "last_name" :   "Fir",
    "age" :         35,
    "about":        "I like to build cabinets",
    "interests":  [ "forestry" ]
}
documentos = [e1, e2, e3]

In [16]:
res = es.index(index='megacorp',doc_type='employee',id=1,body=e1)

In [9]:
res=es.index(index='megacorp',doc_type='employee',id=2,body=e2)
res=es.index(index='megacorp',doc_type='employee',id=3,body=e3)

In [11]:
res.items()

dict_items([('_index', 'megacorp'), ('_type', 'employee'), ('_id', '3'), ('_version', 1), ('result', 'created'), ('_shards', {'total': 2, 'successful': 1, 'failed': 0}), ('_seq_no', 0), ('_primary_term', 1)])

In [13]:
# buscando documento específico
res=es.get(index='megacorp',doc_type='employee',id=3)
print(res)

{'_index': 'megacorp', '_type': 'employee', '_id': '3', '_version': 1, 'found': True, '_source': {'first_name': 'Douglas', 'last_name': 'Fir', 'age': 35, 'about': 'I like to build cabinets', 'interests': ['forestry']}}


In [14]:
# deletando
res=es.delete(index='megacorp',doc_type='employee',id=3)

In [15]:
# operador de metch
res= es.search(index='megacorp',body={'query':{'match':{'first_name':'nitin'}}})
print(res['hits']['hits'])

[{'_index': 'megacorp', '_type': 'employee', '_id': '1', '_score': 0.2876821, '_source': {'first_name': 'nitin', 'last_name': 'panwar', 'age': 27, 'about': 'Love to play cricket', 'interests': ['sports', 'music']}}]


In [16]:
# bool operator:
res= es.search(index='megacorp',body={
        'query':{
            'bool':{
                'must':[{
                        'match':{
                            'first_name':'nitin'
                        }
                    }]
            }
        }
    })

print(res['hits']['hits'])


[{'_index': 'megacorp', '_type': 'employee', '_id': '1', '_score': 0.2876821, '_source': {'first_name': 'nitin', 'last_name': 'panwar', 'age': 27, 'about': 'Love to play cricket', 'interests': ['sports', 'music']}}]


In [19]:
# Full text search
e4={
    "first_name":"asd",
    "last_name":"pafdfd",
    "age": 27,
    "about": "Love to play football",
    "interests": ['sports','music'],
}
res=es.index(index='megacorp',doc_type='employee',id=4,body=e4)

In [21]:
res= es.search(index='megacorp',doc_type='employee',body={
        'query':{
            'match':{
                "about":"play cricket"
            }
        }
    })
for hit in res['hits']['hits']:
    print(hit['_source']['about']) 
    print(hit['_score'])
    print('**********************')

Love to play football
0.7549128
**********************
Love to play cricket
0.5753642
**********************


In [22]:
res= es.search(index='megacorp',doc_type='employee',body={
        "aggs": {
            "all_interests": {
            "terms": { "field": "interests" }
            }
        }
    })

GET http://localhost:9200/megacorp/employee/_search [status:400 request:0.112s]


RequestError: TransportError(400, 'search_phase_execution_exception', 'Fielddata is disabled on text fields by default. Set fielddata=true on [interests] in order to load fielddata in memory by uninverting the inverted index. Note that this can however use significant memory. Alternatively use a keyword field instead.')

In [23]:
import requests

In [24]:
res = requests.get('http://localhost:9200')
print(res.content)

b'{\n  "name" : "_ynoZ8f",\n  "cluster_name" : "elasticsearch",\n  "cluster_uuid" : "QpA-kcsTSx69_qWIbVq5SA",\n  "version" : {\n    "number" : "6.2.4",\n    "build_hash" : "ccec39f",\n    "build_date" : "2018-04-12T20:37:28.497551Z",\n    "build_snapshot" : false,\n    "lucene_version" : "7.2.1",\n    "minimum_wire_compatibility_version" : "5.6.0",\n    "minimum_index_compatibility_version" : "5.0.0"\n  },\n  "tagline" : "You Know, for Search"\n}\n'
